In [ ]:
import joblib
from sklearn.manifold import TSNE
import plotly.express as px
import os
from mair.doc_ids import legal_doc_path_to_id
from mair.data_loading import load_legal_documents_metadata
import pandas as pd
from collections import Counter

In [ ]:
os.chdir('..')

In [ ]:
data = joblib.load('data/processed/docs-bert-embeddings.joblib')

In [ ]:
vectors = []
paths = []
for path, t in data.items():
    vectors.append(t.detach().numpy())
    paths.append(path)
    
X = TSNE(random_state=12).fit_transform(vectors)

In [ ]:
df = pd.DataFrame(X)
df['path']=paths
df['id']= df['path'].apply(legal_doc_path_to_id)

In [ ]:
meta = load_legal_documents_metadata()

In [ ]:
dff =df.merge(meta, on='id')

In [ ]:
dff

In [ ]:
country_mapper=dict()
for country, count in Counter(dff.country).items():
    if count==1:
        country_mapper[country]='other'
    else:
        country_mapper[country]=country

In [ ]:
dff['agg_country'] = dff['country'].map(country_mapper)

In [ ]:
px.scatter(dff, x=0, y=1, color='agg_country', hover_data=['path'])